<a href="https://colab.research.google.com/github/Waynebadu/PL.repo/blob/main/%E7%B0%A1%E6%98%93%E8%A8%98%E5%B8%B3%E8%A1%A8_gradio_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

41371121H 科技117 黃昱瑋
作業1 gradio版本

試算表:[sheet](https://docs.google.com/spreadsheets/d/16Fudzxcy-m2brNoUM6AnELBWeYMm_cGi7TQnUqIUkKc/edit?gid=0#gid=0)

In [6]:
import gradio as gr
import pandas as pd
import datetime
import gspread
from google.colab import auth
from google.auth import default

In [5]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/16Fudzxcy-m2brNoUM6AnELBWeYMm_cGi7TQnUqIUkKc/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

REQUIRED_COLUMNS = ["日期", "類型", "品項", "金額", "支付方式"]

_auth_done = False
_gc = None
_ws = None

In [17]:
def _ensure_auth_and_ws():
    global _auth_done, _gc, _ws
    if not _auth_done:
        # Colab 使用者授權
        auth.authenticate_user()
        creds, _ = default()
        _gc = gspread.authorize(creds)
        _auth_done = True
    if _ws is None:
        gs = _gc.open_by_url(SHEET_URL)
        _ws = gs.worksheet(WORKSHEET_NAME)
        # 確保欄位完整
        _ensure_headers()
    return _ws

def _ensure_headers():
    """確保表頭包含 REQUIRED_COLUMNS；若空表或缺欄，會補齊。"""
    rows = _ws.get_all_values()
    if not rows:
        _ws.append_row(REQUIRED_COLUMNS, value_input_option="USER_ENTERED")
        return
    header = rows[0]
    if header != REQUIRED_COLUMNS:
        # 合併既有欄與必需欄，並以 REQUIRED_COLUMNS 為主順序
        existing = {h: idx for idx, h in enumerate(header)}
        # 若第一列不是必需欄，重寫表頭並搬移資料欄位（簡化處理：補欄位）
        _ws.update('1:1', [REQUIRED_COLUMNS])

        # 若舊資料有相同欄位名，保留；沒有的欄位留空
        if len(rows) > 1:
            new_rows = []
            # Create a mapping from old headers to their index in the old rows
            old_header_map = {h: idx for idx, h in enumerate(header)}
            for r in rows[1:]:
                # Create a new row based on the required columns, using the old data where possible
                new_row_data = []
                for required_col in REQUIRED_COLUMNS:
                    if required_col in old_header_map and old_header_map[required_col] < len(r):
                        new_row_data.append(r[old_header_map[required_col]])
                    else:
                        new_row_data.append("") # Append empty string for missing columns
                new_rows.append(new_row_data)
            # 先清掉舊內容只留表頭
            _ws.resize(rows=1)
            # 再補回資料
            _ws.append_rows(new_rows, value_input_option="USER_ENTERED")


def _read_df():
    ws = _ensure_auth_and_ws()
    values = ws.get_all_values()
    if not values:
        return pd.DataFrame(columns=REQUIRED_COLUMNS)
    df = pd.DataFrame(values[1:], columns=values[0])
    # 型別整理
    if "金額" in df.columns:
        df["金額"] = pd.to_numeric(df["金額"], errors="coerce").fillna(0.0)
    return df

def add_expense(date_str, category, item, amount, payment_method):
    try:
        # 基本驗證
        if not date_str:
            date_str = datetime.date.today().strftime("%Y-%m-%d")
        # 類別/品項/支付方式預設
        category = (category or "未填").strip()
        item = (item or "未填").strip()
        payment_method = (payment_method or "未填").strip()

        # 金額
        try:
            amount_val = float(amount)
        except:
            return "金額格式錯誤，請輸入數字", None, None, None

        ws = _ensure_auth_and_ws()
        # 直接 append 一列
        ws.append_row(
            [date_str, category, item, amount_val, payment_method],
            value_input_option="USER_ENTERED"
        )
        msg = f"✅ 已新增：{date_str}｜{category}｜{item}｜{amount_val}｜{payment_method}"
        # 回傳即時摘要
        df = _read_df()
        cat, settle = _make_summary_tables(df)
        total = float(df["金額"].sum()) if not df.empty else 0.0
        return msg, total, cat, settle
    except Exception as e:
        return f"❌ 新增失敗：{e}", None, None, None

def refresh_summary():
    try:
        df = _read_df()
        if df.empty:
            return "目前沒有資料", 0.0, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        total = float(df["金額"].sum())
        by_cat, settle = _make_summary_tables(df)
        return "✅ 已更新彙總", total, by_cat, settle, df
    except Exception as e:
        return f"❌ 讀取失敗：{e}", 0.0, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

def _make_summary_tables(df: pd.DataFrame):
    # 分類小計
    if "類型" in df.columns:
        by_cat = df.groupby("類型", as_index=False)["金額"].sum().sort_values("金額", ascending=False)
    else:
        by_cat = pd.DataFrame(columns=["類型", "金額"])

    # 付款人結算（AA 分攤）
    if "支付方式" in df.columns and df["支付方式"].notna().any():
        payers = df["支付方式"].replace("", "未填")
        unique_payers = sorted([p for p in payers.unique() if p])
        total = df["金額"].sum()
        n = max(len(unique_payers), 1)
        aa_share = total / n
        paid_by = df.groupby("支付方式", as_index=False)["金額"].sum().rename(columns={"金額": "實付"})
        # 確保每位付款人都有列
        all_rows = pd.DataFrame({"支付方式": unique_payers}).merge(paid_by, on="支付方式", how="left").fillna({"實付": 0.0})
        all_rows["應付(AA)"] = aa_share
        all_rows["差額(實付-應付)"] = all_rows["實付"] - all_rows["應付(AA)"]
        settle = all_rows.sort_values("差額(實付-應付)", ascending=False)
    else:
        settle = pd.DataFrame(columns=["支付方式", "實付", "應付(AA)", "差額(實付-應付)"])
    return by_cat, settle

with gr.Blocks(title="日常支出速算與分攤") as demo:
    gr.Markdown("## 🧾 日常支出速算與分攤（Gradio）\n- 新增支出後自動寫回 Google Sheet\n- 一鍵查看總額、分類小計與 AA 分攤\n- 讀寫工作表：`工作表1`")

    with gr.Tab("➕ 新增支出"):
        with gr.Row():
            date_in = gr.Textbox(label="日期 YYYY-MM-DD", value=datetime.date.today().strftime("%Y-%m-%d"))
            # Removed time_in as it's not in the new required columns
            # time_in = gr.Textbox(label="時間 HH:MM（可留白）", value="")
        with gr.Row():
            cat_in = gr.Textbox(label="類型", placeholder="如 早餐 / 晚餐 / 購物")
            item_in = gr.Textbox(label="品項", placeholder="如 咖啡 / 便當 / 車票")
        with gr.Row():
            amt_in = gr.Textbox(label="金額", placeholder="數字")
            payer_in = gr.Textbox(label="支付方式", placeholder="如 現金 / 信用卡")
        add_btn = gr.Button("新增到工作表")

        add_msg = gr.Markdown()
        total_out = gr.Number(label="目前總額", interactive=False)
        cat_df = gr.Dataframe(label="類型小計", interactive=False)
        settle_df = gr.Dataframe(label="AA 分攤結算", interactive=False)

        add_btn.click(
            fn=add_expense,
            # Updated inputs to match the new function signature
            inputs=[date_in, cat_in, item_in, amt_in, payer_in],
            outputs=[add_msg, total_out, cat_df, settle_df]
        )

    with gr.Tab("📊 彙總 / AA 分攤"):
        refresh_btn = gr.Button("讀取最新彙總")
        msg2 = gr.Markdown()
        total2 = gr.Number(label="總額", interactive=False)
        cat_df2 = gr.Dataframe(label="類型小計", interactive=False)
        settle_df2 = gr.Dataframe(label="AA 分攤結算", interactive=False)
        raw_preview = gr.Dataframe(label="（預覽）最近資料", interactive=False)

        refresh_btn.click(
            fn=refresh_summary,
            inputs=[],
            outputs=[msg2, total2, cat_df2, settle_df2, raw_preview]
        )

    with gr.Tab("📒 檢視原始資料"):
        view_btn = gr.Button("讀取資料")
        view_df = gr.Dataframe(label="全部資料", interactive=False)

        def _view_all():
            try:
                df = _read_df()
                if df.empty:
                    return pd.DataFrame(columns=REQUIRED_COLUMNS)
                return df
            except Exception as e:
                return pd.DataFrame({"錯誤": [str(e)]})

        view_btn.click(fn=_view_all, inputs=[], outputs=[view_df])

# 啟動介面
demo.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>